# Single QPU VQE experiment (Manual Version)

This notebook goes through the running of a VQE algorithm using interlin-q. For simplicity, we only try out using one QPU. This version goes through the calculation of the expectation values manually, by explicitly calling the backend and passing the density matrices and the list of observables.

For an example of how to use the networking protocols to calculate expectation values individually on the different QPUs and then sending back the results to the controller host, check out the other notebooks, with the name "networking".

## Step 1: Import libraries.

First we import all the necessary libraries. Interlin-q is build using the python framework [QuNetSim](https://arxiv.org/abs/2003.06397) which is a python software framework that can be used to simulate quantum networks up to the network layer. We also need PennyLane's chemistry library for decomposing the Hamiltonian. We would also use PennyLane for the optimiser component

In [1]:
%load_ext autoreload
%autoreload 2

# Basic Libraries
import sys
import numpy as np
sys.path.append("../../")

# QuNetSim Components
from qunetsim.components import Network
from qunetsim.objects import Logger
from qunetsim.backends.eqsn_backend import EQSNBackend

# Interlin-q Components
from interlinq import (ControllerHost, Constants, Clock,
Circuit, Layer, ComputingHost, Operation)

# Extra needed components
from hamiltonian_decomposition import decompose

Logger.DISABLED = False

## Step 2: Decompose the Hamiltonian.

In [2]:
geometry = 'h2.xyz'
charge = 0
multiplicity = 1
basis_set = 'sto-3g'
name = 'h2'

In [3]:
coefficients, observables, qubit_num = decompose(name, geometry, charge, multiplicity, basis_set)
terms = list(zip(coefficients, observables))

terms

[(-0.04207897647782276, [('Identity', 0)]),
 (0.17771287465139946, [('PauliZ', 0)]),
 (0.1777128746513994, [('PauliZ', 1)]),
 (-0.24274280513140462, [('PauliZ', 2)]),
 (-0.24274280513140462, [('PauliZ', 3)]),
 (0.17059738328801052, [('PauliZ', 0), ('PauliZ', 1)]),
 (0.04475014401535161,
  [('PauliY', 0), ('PauliX', 1), ('PauliX', 2), ('PauliY', 3)]),
 (-0.04475014401535161,
  [('PauliY', 0), ('PauliY', 1), ('PauliX', 2), ('PauliX', 3)]),
 (-0.04475014401535161,
  [('PauliX', 0), ('PauliX', 1), ('PauliY', 2), ('PauliY', 3)]),
 (0.04475014401535161,
  [('PauliX', 0), ('PauliY', 1), ('PauliY', 2), ('PauliX', 3)]),
 (0.12293305056183798, [('PauliZ', 0), ('PauliZ', 2)]),
 (0.1676831945771896, [('PauliZ', 0), ('PauliZ', 3)]),
 (0.1676831945771896, [('PauliZ', 1), ('PauliZ', 2)]),
 (0.12293305056183798, [('PauliZ', 1), ('PauliZ', 3)]),
 (0.17627640804319591, [('PauliZ', 2), ('PauliZ', 3)])]

## Step 3: Prepare Circuit for Given Parameters

The circuit can be prepared using two different ways: either as one circuit, or several circuits run sequentially. The former approach is simpler and generally better for the optimisation function. The latter is better for debugging and for dynamic components of a quantum circuit (i.e. circuits that have a lot of changing operations).

### Main Blocks

In [4]:
def rotational_gate(params):
    phi, theta, omega = params
    cos = np.cos(theta / 2)
    sin = np.sin(theta / 2)
    
    res = np.array([[np.exp(-1j * (phi + omega) / 2) * cos, -np.exp(1j * (phi - omega) / 2) * sin], 
                     [np.exp(-1j * (phi - omega) / 2) * sin, np.exp(1j * (phi + omega) / 2) * cos]])
    
    return res

In [5]:
def initialisation_operations(q_map):
    ops = []
    host_id = list(q_map.keys())[0]
    
    op = Operation(
        name=Constants.PREPARE_QUBITS,
        qids=q_map[host_id],
        computing_host_ids=[host_id])
    ops.append(op)
    
    ### Workaround for a bug in EQSN
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map[host_id][0], q_map[host_id][1]],
        gate=Operation.CNOT,
        computing_host_ids=[host_id])
    ops.append(op)
    
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map[host_id][0], q_map[host_id][2]],
        gate=Operation.CNOT,
        computing_host_ids=[host_id])
    ops.append(op)
    
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map[host_id][0], q_map[host_id][3]],
        gate=Operation.CNOT,
        computing_host_ids=[host_id])
    ops.append(op)
    ################################
    
    # Prepare the qubits on the computing host
    op = Operation(
        name=Constants.SINGLE,
        qids=[q_map[host_id][0]],
        gate=Operation.X,
        computing_host_ids=[host_id])
    ops.append(op)
    
    op = Operation(
        name=Constants.SINGLE,
        qids=[q_map[host_id][1]],
        gate=Operation.X,
        computing_host_ids=[host_id])
    ops.append(op)
    
    return Layer(ops)

In [6]:
def ansatz_operations(q_map, parameters):
    layers = []
    host_id = list(q_map.keys())[0]
    
    
    ops = []
    
    for i in range(len(q_map[host_id])):
        op = Operation(
            name=Constants.SINGLE,
            qids=[q_map[host_id][i]],
            gate=Operation.CUSTOM,
            gate_param=rotational_gate(parameters[i]),
            computing_host_ids=[host_id])
        
        ops.append(op)
        
    layers.append(Layer(ops))
    
    
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map[host_id][2], q_map[host_id][3]],
        gate=Operation.CNOT,
        computing_host_ids=[host_id])
    
    layers.append(Layer([op]))
    
    
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map[host_id][2], q_map[host_id][0]],
        gate=Operation.CNOT,
        computing_host_ids=[host_id])
    
    layers.append(Layer([op]))
    
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map[host_id][3], q_map[host_id][1]],
        gate=Operation.CNOT,
        computing_host_ids=[host_id])
    
    layers.append(Layer([op]))
    
    return layers

In [7]:
def measurement_operations(q_map):
    ops = []
    # Measuring only the first qubit
    op = Operation(
        name=Constants.MEASURE,
        qids=['q_0_0'],
        cids=['q_0_0'],
        computing_host_ids=[host_id])
    ops.append(op)
    layers.append(Layer(ops))
    
    # Measuring all qubits
    q_ids = q_map[host_id].copy()
    ops = []
    for q_id in q_ids:
        op = Operation(
            name=Constants.MEASURE,
            qids=[q_id],
            cids=[q_id],
            computing_host_ids=[computing_host_ids[0]])
        ops.append(op)
        
    return Layer(ops)

### Single-step approach

In [8]:
def initialise_and_create_ansatz(q_map, parameters):
    layers = []
    host_id = list(q_map.keys())[0]
    
    # Initialise the qubits on the computing host
    layers.append(initialisation_operations(q_map))
    
    # Apply the ansatz
    layers = layers + ansatz_operations(q_map, parameters)
    
    circuit = Circuit(q_map, layers)
    return circuit

In [9]:
def controller_host_protocol(host, q_map, params):
    """
    Protocol for the controller host
    """
    
    monolithic_circuit = initialise_and_create_ansatz(q_map, params)

    host.generate_and_send_schedules(monolithic_circuit)

In [10]:
def computing_host_protocol(host):
    host.receive_schedule()

## Step 4: Run the circuit and get the Expectation Value

In [11]:
def init_network():
    network = Network.get_instance()
    network.delay = 0
    network.start()

    clock = Clock()

    eqsn = EQSNBackend()

    controller_host = ControllerHost(
        host_id="host_1",
        clock=clock, 
        backend=eqsn
    )

    computing_hosts, q_map = controller_host.create_distributed_network(
        num_computing_hosts=1,
        num_qubits_per_host=4)
    controller_host.start()

    network.add_hosts([
        computing_hosts[0],
        controller_host])
    
    return clock, controller_host, computing_hosts, q_map

In [12]:
np.random.seed(0)
params = np.random.normal(0, np.pi, (4, 3))

### Running for the single-step approach

In [13]:
clock, controller_host, computing_hosts, q_map = init_network()

2021-06-12 15:43:34,628: Host QPU_0 started processing
2021-06-12 15:43:34,629: Host host_1 started processing


In [14]:
t1 = controller_host.run_protocol(
    controller_host_protocol,
    (q_map, params))
t2 = computing_hosts[0].run_protocol(computing_host_protocol)

t1.join()
t2.join()

2021-06-12 15:43:35,035: host_1 sends BROADCAST message
2021-06-12 15:43:35,088: sending ACK:1 from QPU_0 to host_1
2021-06-12 15:43:35,115: host_1 received ACK from QPU_0 with sequence number 0
2021-06-12 15:43:35,115: QPU_0 received {"QPU_0": [{"name": "PREPARE_QUBITS", "qids": ["q_0_0", "q_0_1", "q_0_2", "q_0_3"], "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "TWO_QUBIT", "qids": ["q_0_0", "q_0_1"], "cids": null, "gate": "cnot", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "TWO_QUBIT", "qids": ["q_0_0", "q_0_2"], "cids": null, "gate": "cnot", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "TWO_QUBIT", "qids": ["q_0_0", "q_0_3"], "cids": null, "gate": "cnot", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": 

In [15]:
# This should be 7 after the first invocation. 
# For any further invocations, it would increase by 6: 13,19, 25, etc. So is that expected behaviour?
clock.ticks

6

### Using qutip to get the expectation value

In [16]:
from qutip import identity, sigmax, sigmay, sigmaz
from qutip import expect
from qutip import tensor

def string_to_qutip_pauli(string):
    if string == 'Identity':
        return identity(2)
    if string == 'PauliX':
        return sigmax()
    if string == 'PauliY':
        return sigmay()
    if string == 'PauliZ':
        return sigmaz()

def expectation_value(terms, vector, number_of_qubits):
    total = 0

    for term in terms:
        coefficient, observables = term

        obs_tensor_prod = []

        for _ in range(number_of_qubits):
            obs_tensor_prod.append(identity(2))
        
        for obs in observables:
            pauli, index = obs
            
            obs_tensor_prod[index] = string_to_qutip_pauli(pauli)

        obs_tensor_prod = tensor(obs_tensor_prod)
        
        total += coefficient * np.vdot(vector, obs_tensor_prod @ vector) 
        
    return total

In [17]:
indices = []

for qubit_id in computing_hosts[0].qubit_ids:
    indices.append(computing_hosts[0].get_qubit_by_id(qubit_id))

indices

In [18]:
vector = computing_hosts[0].backend.statevector(indices[0])[1]

In [19]:
obser = expectation_value(terms, vector, 4)

obser

(-0.8074697622991748+4.657747522606407e-19j)

## Optimise

In [20]:
def cost_fn(params):
    params = params.reshape(4, 3)
    
    network = Network.get_instance()
    network.delay = 0
    network.start()

    clock = Clock()

    eqsn = EQSNBackend()

    controller_host = ControllerHost(
        host_id="host_1",
        clock=clock, 
        backend=eqsn
    )
    
    qubits = 4

    computing_hosts, q_map = controller_host.create_distributed_network(
        num_computing_hosts=1,
        num_qubits_per_host=qubits)
    controller_host.start()

    network.add_hosts([
        computing_hosts[0],
        controller_host])
    
    t1 = controller_host.run_protocol(
    controller_host_protocol,
    (q_map, params))
    t2 = computing_hosts[0].run_protocol(computing_host_protocol)

    t1.join()
    t2.join()
    
    indices = []

    for qubit_id in computing_hosts[0].qubit_ids:
        indices.append(computing_hosts[0].get_qubit_by_id(qubit_id))
    
    vector = computing_hosts[0].backend.statevector(indices[0])[1]
    
    network.remove_host(computing_hosts[0])
    network.remove_host(controller_host)
    
    return np.real(expectation_value(terms, vector, 4))

In [21]:
np.random.seed(0)
params = np.random.normal(0, np.pi, (4, 3))

params

array([[ 5.54193389,  1.25713095,  3.07479606],
       [ 7.03997361,  5.86710646, -3.07020901],
       [ 2.98479079, -0.47550269, -0.32427159],
       [ 1.28993324,  0.45252622,  4.56873497]])

In [22]:
cost_fn(params)

2021-06-12 15:43:46,576: Host QPU_0 started processing
2021-06-12 15:43:46,576: Host host_1 started processing
2021-06-12 15:43:46,577: host_1 sends BROADCAST message
2021-06-12 15:43:46,671: sending ACK:1 from QPU_0 to host_1
2021-06-12 15:43:46,688: QPU_0 received {"QPU_0": [{"name": "PREPARE_QUBITS", "qids": ["q_0_0", "q_0_1", "q_0_2", "q_0_3"], "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "TWO_QUBIT", "qids": ["q_0_0", "q_0_1"], "cids": null, "gate": "cnot", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "TWO_QUBIT", "qids": ["q_0_0", "q_0_2"], "cids": null, "gate": "cnot", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "TWO_QUBIT", "qids": ["q_0_0", "q_0_3"], "cids": null, "gate": "cnot", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": f

-0.8074697622991748

In [23]:
Logger.DISABLED = True

### SciPy Optimisers

Generally, scipy optimizers are very slow to find the minimum eigenvalue. Thus, we use more specialised optimizers in the next section.

In [27]:
from scipy.optimize import minimize

In [24]:
# This can take up to 5 minutes on an average 2015 laptop
minimum_cobyla = minimize(cost_fn, params, method="COBYLA", tol=1e-3)

minimum_cobyla

     fun: -1.1361893473148623
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 218
  status: 1
 success: True
       x: array([[ 6.76011087e+00, -1.89278921e-04,  3.14012213e+00],
       [ 6.28283752e+00,  5.48607925e+00, -3.29462407e+00],
       [ 2.63003515e+00, -5.50049112e-01, -8.17249049e-05],
       [ 1.05094670e+00,  2.09427402e-01,  4.36005070e+00]])

In [25]:
minimum_cobyla.x.reshape(4,3)

array([[ 6.76011087e+00, -1.89278921e-04,  3.14012213e+00],
       [ 6.28283752e+00,  5.48607925e+00, -3.29462407e+00],
       [ 2.63003515e+00, -5.50049112e-01, -8.17249049e-05],
       [ 1.05094670e+00,  2.09427402e-01,  4.36005070e+00]])

### Scikit-Quant Optimisers

The order of the methods in the cells reflects how well they performs with the same budget. As shown, BOBYQA performs the best with a budget of 100.

In [24]:
from skquant.interop.scipy import *

In [25]:
bounds = np.array([-3*np.pi, 3*np.pi])
bounds = np.tile(bounds, (4*3, 1))

np.random.seed(0)
params = np.random.normal(0, np.pi, (4, 3))

flattened_parameters = params.flatten()
flattened_parameters

array([ 5.54193389,  1.25713095,  3.07479606,  7.03997361,  5.86710646,
       -3.07020901,  2.98479079, -0.47550269, -0.32427159,  1.28993324,
        0.45252622,  4.56873497])

In [28]:
budget = 100
minimum_bobyqa = minimize(cost_fn, flattened_parameters, method=pybobyqa, bounds=bounds, options={'budget' : budget})

minimum_bobyqa

     fun: -1.1329627749839424
 message: 'completed'
    nfev: 100
  status: 0
 success: True
       x: array([ 6.0388495 , -0.09447084,  3.57171167,  6.54305801,  6.27915045,
       -3.56712461,  2.48787519, -0.2274104 ,  0.28697217,  0.79301763,
       -0.01450685,  5.06565057])

In [31]:
budget = 100
minimum_snobfit = minimize(cost_fn, flattened_parameters, method=snobfit, bounds=bounds, options={'budget' : budget})

minimum_snobfit

     fun: -1.0986703464903427
 message: 'completed'
    nfev: 121
  status: 0
 success: True
       x: array([-5.50011192,  0.33891502, -4.96874294, -5.07505444, -6.29650566,
       -4.42851467,  1.59863084, -5.96739241, -3.51657315,  3.89921914,
       -6.50648971, -6.01809772])

In [32]:
budget = 100
minimum_imfil = minimize(cost_fn, flattened_parameters, method=imfil, bounds=bounds, options={'budget' : budget})

minimum_imfil

     fun: -1.016230585889092
 message: 'completed'
    nfev: 107
  status: 0
 success: True
       x: array([ 5.54193389,  5.96951993,  5.43099055,  7.03997361,  5.86710646,
       -3.07020901,  2.98479079, -0.47550269, -0.32427159,  1.28993324,
        0.45252622,  4.56873497])

### Gradient Descent Optimisers 

Gradient-based methods are still not supported by Interlin-q.

In [38]:
%debug
#max_iterations = 200
#conv_tol = 1e-06

#for n in range(max_iterations):
#    params, prev_energy = opt.step_and_cost(cost_fn, params)
#    energy = cost_fn(params)
#    conv = np.abs(energy - prev_energy)

#    if n % 20 == 0:
#        print('Iteration = {:},  Energy = {:.8f} Ha'.format(n, energy))

#    if conv <= conv_tol:
#        break

ERROR:root:No traceback has been produced, nothing to debug.
